<a href="https://colab.research.google.com/github/JerryChenz/InvestmentManagement/blob/master/stock_screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Set inputs

We load the sample dataset from the my github repository, and display the dataset in pandas.

In [28]:
import pandas as pd
summary_url = 'https://raw.githubusercontent.com/JerryChenz/InvestmentManagementOpen/main/financial_models/Opportunities/Screener/screener_summary.csv'
df = pd.read_csv(summary_url)
df = df
df = df.fillna(0)
display(df)

,Ticker,Name,Exchange,Price,Price_currency,Shares,Reporting_Currency,Fx_rate,Dividend,Buyback,...,EBIT,EbitMargin,Avg_ebit_margin,Avg_ebit_growth,InterestExpense,NetIncomeCommonStockholders,NetMargin,Avg_net_margin,Avg_NetIncome_growth,Years_of_data
0,000036.SZ,CHINA UNION HOLD L,SHZ,4.0100,CNY,1.483930e+09,CNY,1.000000,0.000000,0.000000,...,1.034388e+09,54.96,61.500000,-30.57,5.205453e+07,4.063153e+08,21.59,22.590000,-29.43,3.0
1,000045.SZ,SHN TEXTILE HLDGS,SHZ,10.6700,CNY,4.570220e+08,CNY,1.000000,0.000000,0.000000,...,3.237886e+08,14.12,10.793333,53.97,1.430628e+07,6.116238e+07,2.67,1.783333,76.74,3.0
2,000088.SZ,SHN YAN TIAN PORT,SHZ,4.9900,CNY,2.249160e+09,CNY,1.000000,0.000000,0.000000,...,2.232715e+08,32.84,31.630000,15.78,7.878430e+07,4.614137e+08,67.87,67.400000,13.39,3.0
3,000096.SZ,SHENZHEN GUANGJU E,SHZ,8.5000,CNY,5.280000e+08,CNY,1.000000,0.000000,0.000000,...,1.078495e+08,6.66,6.390000,7.97,1.005905e+05,6.770438e+07,4.18,7.483333,-21.90,3.0
4,000099.SZ,CITIC OFFSHORE HEL,SHZ,7.2800,CNY,7.757700e+08,CNY,1.000000,0.000000,0.000000,...,4.122275e+08,24.53,25.100000,5.79,6.176388e+07,2.464817e+08,14.67,14.056667,8.07,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2582,ZD,"Ziff Davis, Inc.",NMS,82.8000,USD,4.719160e+07,USD,1.000000,0.000000,1.659766,...,2.788430e+08,19.68,23.540000,-4.88,7.202300e+07,4.963880e+08,35.04,20.266667,100.26,3.0
2583,ZG,"Zillow Group, Inc.",NMS,39.6300,USD,5.819710e+07,USD,1.000000,0.000000,5.194365,...,2.281910e+08,2.80,9.256667,40.67,1.919100e+08,-5.277770e+08,-6.48,-7.486667,89.32,3.0
2584,ZIM,ZIM Integrated Shipping Service,NYQ,17.5849,USD,1.200470e+08,USD,1.000000,4.468150,0.000000,...,5.836800e+09,54.40,25.720000,554.54,1.688980e+08,4.640305e+09,43.25,18.560000,-1079.03,3.0
2585,ZTO,ZTO Express (Cayman) Inc.,NYQ,28.2000,USD,8.097330e+08,CNY,0.147437,1.672118,4.705978,...,4.713508e+09,15.50,18.333333,-2.41,1.265030e+08,4.754827e+09,15.64,19.466667,-6.87,3.0


In [29]:
# capitalization in reporting currency
capitalization_price = df['Price'] * df['Shares']
capitalization_report = capitalization_price * df['Fx_rate']
total_debt = df['CurrentDebtAndCapitalLeaseObligation'] + df['LongTermDebtAndCapitalLeaseObligation']
# more easily realizable non-operating assets
monetary_assets = df['CashAndCashEquivalents'] + df['OtherShortTermInvestments'] + df['InvestmentinFinancialAssets']
# less easily realizable non-operating assets
fixed_nonop_assets = (df['InvestmentProperties'] + df['LongTermEquityInvestment']) * 0.5
total_nonop_assets = monetary_assets + fixed_nonop_assets
enterprise_value = capitalization_report + total_debt + df['MinorityInterest'] - df['CashAndCashEquivalents'] - total_nonop_assets
# liquidity_coverage_ratio
lcr = monetary_assets / df['CurrentLiabilities']
current_ratio = df['CurrentAssets'] / df['CurrentLiabilities']
# dividend rate & buyback rate
dividend_rate = df['Dividend'] / df['Price']
buyback_rate = df['Buyback'] / df['Price']
# Net PPE/ Sales
ppe_multiple = df['NetPPE'] / df['TotalRevenue']

# Step 2: Screening Criteria

We can screen using different sets of conditions, then merge them later.

In [30]:
# common fitlering conditions
common_1 = df['CashAndCashEquivalents'] > df['CurrentDebtAndCapitalLeaseObligation']
common_2 = lcr >= 0.8
common_3 = (total_debt / df['CurrentAssets']) < 1.5
common_4 = df['Avg_Gross_margin'] > 10
common_5 = capitalization_price > 1000000000

In [31]:
# 1st set of conditions
condition_1 = capitalization_price > 6000000000
condition_2 = df['Avg_ebit_margin'] > 15

In [32]:
# 2nd set of conditions
condition_5 = capitalization_price <= 6000000000
condition_6 = dividend_rate > 0.01
condition_7 = buyback_rate > 0.01

#Step 3. Output

Filter the dataset using the above conditions

In [33]:
# filtered by common conditions
common_df = df
common_df['EV'] = enterprise_value
common_df['Dividend rate'] = dividend_rate
common_df['Buyback rate'] = buyback_rate
common_df = common_df.loc[common_1 & common_2 & common_3 & common_4]
display(common_df)

,Ticker,Name,Exchange,Price,Price_currency,Shares,Reporting_Currency,Fx_rate,Dividend,Buyback,...,Avg_ebit_growth,InterestExpense,NetIncomeCommonStockholders,NetMargin,Avg_net_margin,Avg_NetIncome_growth,Years_of_data,EV,Dividend rate,Buyback rate
0,000036.SZ,CHINA UNION HOLD L,SHZ,4.010,CNY,1.483930e+09,CNY,1.000000,0.000000,0.000000,...,-30.57,5.205453e+07,4.063153e+08,21.59,22.590000,-29.43,3.0,8.968230e+08,0.000000,0.000000
1,000045.SZ,SHN TEXTILE HLDGS,SHZ,10.670,CNY,4.570220e+08,CNY,1.000000,0.000000,0.000000,...,53.97,1.430628e+07,6.116238e+07,2.67,1.783333,76.74,3.0,3.745815e+09,0.000000,0.000000
2,000088.SZ,SHN YAN TIAN PORT,SHZ,4.990,CNY,2.249160e+09,CNY,1.000000,0.000000,0.000000,...,15.78,7.878430e+07,4.614137e+08,67.87,67.400000,13.39,3.0,8.186662e+09,0.000000,0.000000
3,000096.SZ,SHENZHEN GUANGJU E,SHZ,8.500,CNY,5.280000e+08,CNY,1.000000,0.000000,0.000000,...,7.97,1.005905e+05,6.770438e+07,4.18,7.483333,-21.90,3.0,1.208206e+09,0.000000,0.000000
4,000099.SZ,CITIC OFFSHORE HEL,SHZ,7.280,CNY,7.757700e+08,CNY,1.000000,0.000000,0.000000,...,5.79,6.176388e+07,2.464817e+08,14.67,14.056667,8.07,3.0,2.969887e+09,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2577,YMM,Full Truck Alliance Co. Ltd.,NYQ,9.425,USD,1.108400e+09,CNY,0.147437,0.000000,2.332585,...,254.89,4.000000e+04,-4.172880e+09,-89.60,-96.776667,75.94,3.0,-3.091314e+10,0.000000,0.247489
2580,YY,JOYY Inc.,NMS,36.965,USD,7.084300e+07,USD,1.000000,0.000000,5.627049,...,-174.94,1.447500e+07,-8.952900e+07,-3.42,27.470000,49.21,3.0,-1.618068e+09,0.000000,0.152226
2582,ZD,"Ziff Davis, Inc.",NMS,82.800,USD,4.719160e+07,USD,1.000000,0.000000,1.659766,...,-4.88,7.202300e+07,4.963880e+08,35.04,20.266667,100.26,3.0,3.668737e+09,0.000000,0.020045
2583,ZG,"Zillow Group, Inc.",NMS,39.630,USD,5.819710e+07,USD,1.000000,0.000000,5.194365,...,40.67,1.919100e+08,-5.277770e+08,-6.48,-7.486667,89.32,3.0,-1.280649e+09,0.000000,0.131072


In [34]:
# fitlered by 1st set of conditions
df_1 = common_df
df_1 = df_1.loc[condition_1 & condition_2]
print(df_1.to_string())

         Ticker                             Name Exchange     Price Price_currency        Shares Reporting_Currency   Fx_rate  Dividend    Buyback       Last_fy   TotalAssets  CurrentAssets  CurrentLiabilities  CurrentDebtAndCapitalLeaseObligation  CurrentCapitalLeaseObligation  LongTermDebtAndCapitalLeaseObligation  LongTermCapitalLeaseObligation  TotalEquityGrossMinorityInterest  MinorityInterest  CashAndCashEquivalents  OtherShortTermInvestments  InvestmentProperties  LongTermEquityInvestment  InvestmentinFinancialAssets        NetPPE  TotalRevenue  Avg_sales_growth  CostOfRevenue  GrossMargin  Avg_Gross_margin  SellingGeneralAndAdministration          EBIT  EbitMargin  Avg_ebit_margin  Avg_ebit_growth  InterestExpense  NetIncomeCommonStockholders  NetMargin  Avg_net_margin  Avg_NetIncome_growth  Years_of_data            EV  Dividend rate  Buyback rate
2     000088.SZ                SHN YAN TIAN PORT      SHZ    4.9900            CNY  2.249160e+09                CNY  1.000000  0.000

In [35]:
# filtered by 2nd set of conditions
df_2 = common_df
df_2 = df_2.loc[condition_5 & (condition_6 | condition_7)]
print(df_2.to_string())

       Ticker                             Name Exchange     Price Price_currency        Shares Reporting_Currency   Fx_rate  Dividend    Buyback       Last_fy   TotalAssets  CurrentAssets  CurrentLiabilities  CurrentDebtAndCapitalLeaseObligation  CurrentCapitalLeaseObligation  LongTermDebtAndCapitalLeaseObligation  LongTermCapitalLeaseObligation  TotalEquityGrossMinorityInterest  MinorityInterest  CashAndCashEquivalents  OtherShortTermInvestments  InvestmentProperties  LongTermEquityInvestment  InvestmentinFinancialAssets        NetPPE  TotalRevenue  Avg_sales_growth  CostOfRevenue  GrossMargin  Avg_Gross_margin  SellingGeneralAndAdministration          EBIT  EbitMargin  Avg_ebit_margin  Avg_ebit_growth  InterestExpense  NetIncomeCommonStockholders  NetMargin  Avg_net_margin  Avg_NetIncome_growth  Years_of_data            EV  Dividend rate  Buyback rate
241   0032.HK                  CROSS-HAR(HOLD)      HKG   10.8000            HKD  3.726880e+08                HKD  1.000000  0.420510 

In [36]:
# combine the results
result_set = pd.concat([df_1, df_2])
result_set = result_set.groupby("Ticker").first()
result_set = result_set.sort_values(by=['EV', 'Dividend rate'], ascending=[True, False]).reset_index()
print(result_set.to_string())

        Ticker                             Name Exchange     Price Price_currency        Shares Reporting_Currency   Fx_rate  Dividend    Buyback       Last_fy   TotalAssets  CurrentAssets  CurrentLiabilities  CurrentDebtAndCapitalLeaseObligation  CurrentCapitalLeaseObligation  LongTermDebtAndCapitalLeaseObligation  LongTermCapitalLeaseObligation  TotalEquityGrossMinorityInterest  MinorityInterest  CashAndCashEquivalents  OtherShortTermInvestments  InvestmentProperties  LongTermEquityInvestment  InvestmentinFinancialAssets        NetPPE  TotalRevenue  Avg_sales_growth  CostOfRevenue  GrossMargin  Avg_Gross_margin  SellingGeneralAndAdministration          EBIT  EbitMargin  Avg_ebit_margin  Avg_ebit_growth  InterestExpense  NetIncomeCommonStockholders  NetMargin  Avg_net_margin  Avg_NetIncome_growth  Years_of_data            EV  Dividend rate  Buyback rate
0         BABA    Alibaba Group Holding Limited      NYQ  114.8550            USD  2.647540e+09                CNY  0.147437  0.00000